In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [3]:
train_generator = train_datagen.flow_from_directory(
    '/Users/tarunreddy/Desktop/dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Use the 80% for training
)

validation_generator = train_datagen.flow_from_directory(
    '/Users/tarunreddy/Desktop/dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Use the 20% for validation
)

Found 4291 images belonging to 2 classes.
Found 1072 images belonging to 2 classes.


In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [5]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
# Reduce learning rate when the validation accuracy plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',  # Monitor validation accuracy
    factor=0.5,  # Reduce the learning rate by a factor of 0.5
    patience=3,  # Number of epochs with no improvement to wait before reducing LR
    min_lr=1e-6,  # Minimum learning rate
    verbose=1
)

# Stop training when the validation accuracy has stopped improving
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy
    patience=3,  # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True,  # Restore model weights from the epoch with the best accuracy
    verbose=1
)

# Create a checkpoint callback to save the best model
checkpoint = ModelCheckpoint(
    'best_model.keras',  # The file path to save the model
    monitor='val_accuracy',  # The metric to monitor (use 'val_loss' if preferred)
    save_best_only=True,  # Save only the model with the best metric score
    mode='max',  # 'max' if monitoring accuracy, 'min' if monitoring loss
    verbose=1  # Print messages when saving the model
)

In [8]:
# Recompile the model for fine-tuning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=20,  # Number of epochs
    steps_per_epoch=len(train_generator),  # Adjust based on your dataset size
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 0.8705 - loss: 6.1034
Epoch 1: val_accuracy improved from -inf to 0.85541, saving model to best_model.keras
135/135 ━━━━━━━━━━━━━━━━━━━━ 65s 459ms/step - accuracy: 0.8710 - loss: 6.0819 - val_accuracy: 0.8554 - val_loss: 0.8932 - learning_rate: 0.0010
Epoch 2/20
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/20


2024-09-28 21:05:21.838582: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-09-28 21:05:21.852920: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/opt/anaconda3/lib/python3.12/site-packages/kera

135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.9627 - loss: 0.4665
Epoch 3: val_accuracy did not improve from 0.85541
135/135 ━━━━━━━━━━━━━━━━━━━━ 62s 448ms/step - accuracy: 0.9627 - loss: 0.4658 - val_accuracy: 0.8274 - val_loss: 0.5604 - learning_rate: 0.0010
Epoch 4/20
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 116us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/20


2024-09-28 21:06:24.279008: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.9712 - loss: 0.1999
Epoch 5: val_accuracy did not improve from 0.85541
135/135 ━━━━━━━━━━━━━━━━━━━━ 62s 447ms/step - accuracy: 0.9711 - loss: 0.1999 - val_accuracy: 0.8302 - val_loss: 0.4771 - learning_rate: 0.0010
Epoch 6/20
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 94us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 7/20
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.9694 - loss: 0.1651
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 7: val_accuracy did not improve from 0.85541
135/135 ━━━━━━━━━━━━━━━━━━━━ 62s 446ms/step - accuracy: 0.9693 - loss: 0.1652 - val_accuracy: 0.8321 - val_loss: 0.4994 - learning_rate: 0.0010
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 1.


In [9]:
# Load the best saved model
best_model = load_model('best_model.keras')

# Evaluate the model on the validation set
val_loss, val_accuracy = best_model.evaluate(validation_generator, steps=len(validation_generator), verbose=1)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 495ms/step - accuracy: 0.8413 - loss: 0.8908
Validation Loss: 0.8859252333641052
Validation Accuracy: 0.8516790866851807


In [48]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to the image you want to predict
img_path = '/Users/tarunreddy/Desktop/images/light_spoil.webp'  # Replace with your image path

# Load and preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0  # Rescale pixel values
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for batch size

# Make the prediction
prediction = model.predict(img_array)
# Convert probability to class
predicted_class = (prediction > 0.6).astype(int)

# Output the prediction
if predicted_class[0] == 0:
    print("The image is classified as: Fresh")
else:
    print("The image is classified as: Spoiled")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.9944652]]
The image is classified as: Spoiled
